In [1]:
# AWS Rekognition to get bbox
import numpy as np
import boto3
from PIL import Image
from utils.rekognition import determine_color, draw_animal_count
import cv2
import time
import math
import os
import io
import json
import pandas as pd
from utils.config import *
from utils.fix_annotation import *
from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

In [2]:
def get_kp_color(label):
    # BGR
    color = (0, 0, 255)
    if label == 'Head':
        color = ['#EC51F8', '#74F54B', 
                 '#EC51F8', '#74F54B',
                 '#4394F9', '#F49736',
                 '#F49736', '#FFFB56',
                 '#FFFB56', '#4394F9',
                 '#07178D']
    elif label == 'Spine':
          color = ['#4394F9', '#4394F9', '#4394F9',
                  '#4394F9', '#4394F9', '#4394F9', 
                  '#4394F9', '#4394F9', '#24518D']
    elif label == 'Tail':
        color = ['#EC51F8', '#EC51F8',
                '#EC51F8', '#EC51F8',
                '#EC51F8', '#892B8E']
    elif label == 'Leg_front':
        color = ['#F49736', '#F49736',
                '#F49736', '#F49736',
                '#F49736', '#F49736',
                '#F49736', '#F49736',
                '#F49736', '#8C551E']
    elif label == 'Leg_back':
        color = ['#74F54B', '#74F54B',
                '#74F54B', '#74F54B',
                '#74F54B', '#74F54B',
                '#74F54B', '#74F54B',
                '#74F54B', '#3F8D28',]
    return color

def get_skeleton(label):
    skeleton_list = []
    if label == 'Head':
        skeleton_list = [[4, 0], [4, 2], [0, 2], [1, 3], 
                        [5, 6], [7, 8], [0, 1], [1, 5],
                        [5, 7], [7, 9], [2, 3], [3, 6],
                        [6, 8], [8, 9], [4, 9]]
    elif label == 'Spine':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4],
                        [4, 5], [5, 6], [6, 7]]
    elif label == 'Tail':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4]]
    elif label == 'Leg_front':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4],
                        [4, 5], [5, 6], [6, 7], [7, 8]]
    elif label == 'Leg_back':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4],
                        [4, 5], [5, 6], [6, 7], [7, 8]]
    return skeleton_list

def get_kp_name(label, i):
    name_list = []
    if label == 'Head':
        name_list = ["Ear_Tip_Ri", "Ear_Base_R", "Ear_Tip_Le", "Ear_Base_L", 
                     "Head_Top", "Eye_Right", "Eye_Left", "Nostril_Ri", 
                     "Nostril_Le", "Mouth_Bott"]
    elif label == 'Spine':
        name_list = ["Head", "Neck", "Shoulder F", "Arch 1", 
                     "Arch Mid", "Arch 3", "Should Bac", "Tail"]
    elif label == 'Tail':
        name_list = ["Tail Start", "Tail 1/4", "Tail half", "Tail 3/4", "Tail End"]
    elif label == 'Leg_front':
        name_list = ["Right Hoof", "Right Ankl", "Right Knee", "Right Hip", 
                     "Shoulder", "Left Hip", "Left Knee", "Left Ankle", 
                     "Left Hoof"]
    elif label == 'Leg_back':
        name_list = ["Right Hoof", "Right Ankle", "Right Knee", "Right Hip", 
                     "Shoulder", "Left Hip", "Left Knee", "Left Ankle", 
                     "Left Hoof"]
    return name_list[i]

# def get_font(font_str):
#     font = cv2.FONT_HERSHEY_COMPLEX
#     if font_str == "cv2.FONT_HERSHEY_COMPLEX":
        
#     return font

In [3]:
def get_shift(radius, angle):
    x = radius * math.cos(math.radians(angle))
    y = radius * math.sin(math.radians(angle))
    return x, y

In [4]:
def get_angle(a, b, c):
    ang = math.degrees(math.atan2(
        c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    
    return ang + 360 if ang < 0 else ang

In [5]:
def get_all_angle(point_1, point_2, point_3, filp):
    angle = 360 - get_angle(point_1, point_2, point_3)
    # pieslice bbox
    delta_x = point_3[0] - point_2[0]
    delta_y = point_3[1] - point_2[1]
    if filp:
        # return to original angle
        angle = -1 * angle + 360
        delta_x = point_1[0] - point_2[0]
        delta_y = point_1[1] - point_2[1]
    start_angle = math.degrees(math.atan2(delta_y, delta_x))
    end_angle = start_angle + round(angle)
    mid_angle = start_angle + round(angle) / 2
    return angle, start_angle, end_angle, mid_angle

In [6]:
def draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, fill_color, broder_color, fill_opacity, opacity_adjust=0, radius=40):
    # draw filled angle
    shapes = np.zeros_like(img, np.uint8)
#     cv2.ellipse(shapes, (int(x+shift_list[0]), int(y+shift_list[1])), (radius, radius), 0, int(angle_list[1]), int(angle_list[2]), fill_color, cv2.FILLED)
    cv2.ellipse(shapes, (int(x + shift_list[0]), int(y + shift_list[1])), 
                (radius, radius), 0, angle_list[1], angle_list[2], 
                fill_color, cv2.FILLED)
    mask = shapes.astype(bool)
    img[mask] = cv2.addWeighted(img, 1 - fill_opacity + opacity_adjust, 
                                shapes, fill_opacity, 0)[mask]
    # edge
    cv2.ellipse(img, (int(x+shift_list[0]), int(y + shift_list[1])), (radius, radius), 0, 
                angle_list[1], angle_list[2], broder_color, 2)
    # draw start angle outline
    start_point = (int(x + shift_list[0]), int(y + shift_list[1]))
    end_point = (int(x + shift_list[0] + radius * math.cos(math.radians(angle_list[1]))),
                 int(y + shift_list[1] + radius * math.sin(math.radians(angle_list[1]))))
    cv2.line(img, start_point, end_point, broder_color, 2)
    # draw end angle outline
    end_point = (int(x + shift_list[0] + radius * math.cos(math.radians(angle_list[2]))),
                 int(y + shift_list[1] + radius * math.sin(math.radians(angle_list[2]))))
    cv2.line(img, start_point, end_point, broder_color, 2)
    # draw full angle
    angle_str = "{:.0f}".format(angle_list[0])
    angle_str_len = len(angle_str)
    cv2.putText(img, angle_str, (int(x+shift_list[0])+shift_text_list[0], int(y)+shift_text_list[1]), 
                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.ellipse(img, (int(x+shift_list[0]+shift_text_list[0]+angle_str_len*12), int(y-8)+shift_text_list[1]), 
                (3, 3), 0, 0, 360, (255, 255, 255), 1)

In [7]:
def vis_pose(points, face_right, img, label, frameId, angle_list, kp_config, angle_config, CS_THR=0.4):
    # convert df to list
    points = [i[-3:] for i in points.values.tolist()]
    # keypoints
    kp_color = kp_config[label]["filled_color"]
#     kp_color = get_kp_color(label)
    # connect line
    skeleton_list = get_skeleton(label)
    for ske in skeleton_list:
        fir_pt_x, fir_pt_y, fir_pt_p = points[ske[0]]
        sec_pt_x, sec_pt_y, sec_pt_p = points[ske[1]]
        if fir_pt_p > CS_THR and sec_pt_p > CS_THR:
            # draw line
            start_point = (int(fir_pt_x), int(fir_pt_y))
            end_point = (int(sec_pt_x), int(sec_pt_y))
            bgr_color = hex_to_bgr(kp_config[label]["line_color"])
            cv2.line(img, start_point, end_point, bgr_color, kp_config[label]["line_width"])
    
    for i, point in enumerate(points):
        x, y, p = point
        if p > CS_THR:
            x = int(x)
            y = int(y)
            bgr_color = hex_to_bgr(kp_color[i])
            point_width = kp_config[label]["point_width"]
            # draw kp
            cv2.ellipse(img, (x, y), (point_width, point_width), 0, 0, 360, bgr_color, cv2.FILLED)
            # draw kp outline
            bgr_color = hex_to_bgr(kp_config[label]["line_color"])
            cv2.ellipse(img, (x, y), (point_width+2, point_width+2), 0, 0, 360, bgr_color, 2)
            if kp_config[label]["show_kp_name"]:
                kp_name = get_kp_name(label, i)
                cv2.putText(img, kp_name, (x-40, y-40), 
                            kp_config[label]['font'], kp_config[label]['font_size'], (255, 255, 255), 
                            1, cv2.LINE_AA)
            if kp_config[label]["show_conf"]:
                cv2.putText(img, "conf: {:.1f}%".format(p*100), (x-40, y-20), 
                            kp_config[label]['font'], kp_config[label]['font_size'], (255, 255, 255), 
                            1, cv2.LINE_AA)
            # draw index
            if kp_config[label]["show_idx"]:
                cv2.putText(img, str(i), (x-20, y-20), 
                            cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 0), 
                            1, cv2.LINE_AA)
            #***init keypoint data
#             bbox_type = get_bbox_type(label)
            kp_data_list = [frameId, get_kp_name(label, i), x, y, 0]
            #***
            # draw spine angle
            if (label == 'Spine') and i > 0 and i < 7 and points[i-1][2] > CS_THR and points[i+1][2] > CS_THR:
                delta_x = points[i-1][0] - points[i][0]
                angle, start_angle, end_angle, mid_angle = get_all_angle(points[i+1], points[i], points[i-1], delta_x > 0)
                kp_data_list[-1] = angle
                shift_text_list = [-20, -22]
                shift_x, shift_y = get_shift(10, mid_angle)
                shift_list = [shift_x, shift_y]
                angle_list = [angle, start_angle, end_angle, mid_angle]
                draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                  hex_to_bgr(angle_config[label]['filled_color']), hex_to_bgr(angle_config[label]['border_color']), angle_config[label]['opacity'])
            # draw ear angle
            elif (label == 'Head'):
                # draw angle for point 1 of head
                if i == 1 and points[0][2] > CS_THR and points[1][2] > CS_THR and points[3][2] > CS_THR:
                    # cacluate the angle of opposite side
                    delta_x = points[3][0] - points[1][0]
                    angle, start_angle, end_angle, mid_angle = get_all_angle(points[0], points[1], points[3], delta_x > 0)
                    kp_data_list[-1] = angle
                    shift_text_list = [-15, -15]
                    shift_x, shift_y = get_shift(10, mid_angle)
                    shift_list = [shift_x, shift_y]
                    angle_list = [angle, start_angle, end_angle, mid_angle]
                    draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                      hex_to_bgr(angle_config[label]['filled_color']), hex_to_bgr(angle_config[label]['border_color']), angle_config[label]['opacity'])
                # draw angle for point 3 of head
                if i == 3 and points[1][2] > CS_THR and points[2][2] > CS_THR and points[3][2] > CS_THR:
                    # cacluate the angle of opposite side
                    delta_x = points[2][0] - points[3][0]
                    angle, start_angle, end_angle, mid_angle = get_all_angle(points[1], points[3], points[2], delta_x > 0)
                    kp_data_list[-1] = angle
                    shift_text_list = [-15, -15]
                    shift_x, shift_y = get_shift(10, mid_angle)
                    shift_list = [shift_x, shift_y]
                    angle_list = [angle, start_angle, end_angle, mid_angle]
                    draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                      hex_to_bgr(angle_config[label]['filled_color']), hex_to_bgr(angle_config[label]['border_color']), angle_config[label]['opacity'])
            # draw tail angle
            elif (label == 'Tail') and i > 0 and i < 4 and points[i-1][2] > CS_THR and points[i+1][2] > CS_THR and points[0][2] > CS_THR and points[1][2] > CS_THR:
                filp = (points[0][0] - points[1][0] > 0)
                angle, start_angle, end_angle, mid_angle = get_all_angle(points[i+1], points[i], points[i-1], filp)
                kp_data_list[-1] = angle
                shift_text_list = [0, 5]
                shift_x, shift_y = get_shift(10, mid_angle)
                shift_list = [shift_x, shift_y]
                angle_list = [angle, start_angle, end_angle, mid_angle]
                if filp:
                    shift_text_list[0] = -40
                draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                  hex_to_bgr(angle_config[label]['filled_color']), hex_to_bgr(angle_config[label]['border_color']), angle_config[label]['opacity'])
            # draw leg angle
            elif (label == 'Leg_front' or label == 'Leg_back') and i > 0 and i < 8 and points[i-1][2] > CS_THR and points[i+1][2] > CS_THR:
                # filp condition
                face_right_2 = points[5][0] < points[3][0]
                filp = (i == 4 and not face_right_2) or (i >= 5 and face_right) or (i < 4 and not face_right)
                # get all angles
                angle, start_angle, end_angle, mid_angle = get_all_angle(points[i+1], points[i], points[i-1], filp)
                shift_text_list = [0, 5]
                shift_dis = 10
                # diff text shift cases
                if i == 4:
                    shift_text_list[0] = 12
                    shift_dis = 15
                if i == 1 or i == 7:
                    shift_text_list[1] = 10
                if i != 4 and face_right:
                    shift_text_list[0] = -40
                # calculate shift position
                shift_x, shift_y = get_shift(shift_dis, mid_angle)
                shift_list = [shift_x, shift_y]
                angle_list = [angle, start_angle, end_angle, mid_angle]
                draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                      hex_to_bgr(angle_config[label]['filled_color']), hex_to_bgr(angle_config[label]['border_color']), angle_config[label]['opacity'], opacity_adjust=0.2)
            angle_list.append(kp_data_list)
    return img

In [8]:
def vis_mutual_angle(pred_1, pred_2, label_1, label_2, img, frameId, data_list, angle_config, CS_THR=0.4):
    pred_1_idx = 4
    if label_1 == 'Head' and label_2 == 'Spine':
        x1, y1, p1 = pred_1[pred_1['kp_idx']==pred_1_idx].values.tolist()[0][-3:]
        x2, y2, p2 = pred_2[pred_2['kp_idx']==0].values.tolist()[0][-3:]
        x3, y3, p3 = pred_1[pred_1['kp_idx']==9].values.tolist()[0][-3:]
        x4, y4, p4 = pred_2[pred_2['kp_idx']==1].values.tolist()[0][-3:]
        color_code = angle_config['Spine']['filled_color']
        border_color_code = angle_config['Spine']['border_color']
        opacity = angle_config['Spine']['opacity']
    elif label_1 == 'Tail' and label_2 == 'Spine':
        pred_1_idx = 0
        x1, y1, p1 = pred_1[pred_1['kp_idx']==pred_1_idx].values.tolist()[0][-3:]
        x2, y2, p2 = pred_2[pred_2['kp_idx']==7].values.tolist()[0][-3:]
        x3, y3, p3 = pred_1[pred_1['kp_idx']==1].values.tolist()[0][-3:]
        x4, y4, p4 = pred_2[pred_2['kp_idx']==6].values.tolist()[0][-3:]
        color_code = angle_config['Tail']['filled_color']
        border_color_code = angle_config['Tail']['border_color']
        opacity = angle_config['Tail']['opacity']
    else:
        print('no matched label')
        return img

    if p3 <= CS_THR or p4 <= CS_THR or (p1 <= CS_THR and p2 <= CS_THR):
        return img
    if p1 > CS_THR and p2 <= CS_THR:
        x2, y2, p2 = x1, y1, p1
    elif p2 > CS_THR and p1 <= CS_THR:
        x1, y1, p1 = x2, y2, p2
    bbox_type_1 = get_bbox_type(label_1)
    bbox_type_2 = get_bbox_type(label_2)
    mid_point_x = int((x1 + x2) / 2)
    mid_point_y = int((y1 + y2) / 2)
    # draw angle
    filp = (x4-x2 > 0)
    angle, start_angle, end_angle, mid_angle = get_all_angle([x3, y3], [mid_point_x, mid_point_y], [x4, y4], filp)
    kp_data_list = [frameId, get_kp_name(label_1, pred_1_idx), mid_point_x, mid_point_y, 0]
    kp_data_list[-1] = angle
    shift_text_list = [5, -8]
    shift_x, shift_y = get_shift(10, mid_angle)
    shift_list = [shift_x, shift_y]
    angle_list = [angle, start_angle, end_angle, mid_angle]
    if filp:
        shift_text_list[0] = -25
    draw_filled_angle(img, mid_point_x, mid_point_y, angle_list, shift_list, shift_text_list, 
                      hex_to_bgr(color_code), hex_to_bgr(border_color_code), opacity)

    data_list.append(kp_data_list)
    return img

In [9]:
def draw_line_connect(pred_1, pred_2, label_1, label_2, img, kp_config, CS_THR=0.4):
    if label_1 == 'Spine' and label_2 == 'Leg_front':
        # keypoint color
        line_color = kp_config[label_1]['line_color']
        spine_kp_idx = 2
        legf_kp_idx = 4
        fir_pt = pred_1[pred_1['kp_idx']==spine_kp_idx]
        sec_pt = pred_2[pred_2['kp_idx']==legf_kp_idx]
        fir_pt_x, fir_pt_y, fir_pt_p = fir_pt['x'].values[0], fir_pt['y'].values[0], fir_pt['conf'].values[0]
        sec_pt_x, sec_pt_y, sec_pt_p = sec_pt['x'].values[0], sec_pt['y'].values[0], sec_pt['conf'].values[0]
        if fir_pt_p > CS_THR and sec_pt_p > CS_THR:
            # draw line
            start_point = (int(fir_pt_x), int(fir_pt_y))
            end_point = (int(sec_pt_x), int(sec_pt_y))
            bgr_color = hex_to_bgr(line_color)
            cv2.line(img, start_point, end_point, bgr_color, 5)
    elif label_1 == 'Spine' and label_2 == 'Leg_back':
        # keypoint color
        line_color = kp_config[label_1]['line_color']
        spine_kp_idx = len(pred_1)-2
        legb_kp_idx = 4
        fir_pt = pred_1[pred_1['kp_idx']==spine_kp_idx]
        sec_pt = pred_2[pred_2['kp_idx']==legb_kp_idx]
        fir_pt_x, fir_pt_y, fir_pt_p = fir_pt['x'].values[0], fir_pt['y'].values[0], fir_pt['conf'].values[0]
        sec_pt_x, sec_pt_y, sec_pt_p = sec_pt['x'].values[0], sec_pt['y'].values[0], sec_pt['conf'].values[0]
        if fir_pt_p > CS_THR and sec_pt_p > CS_THR:
            # draw line
            start_point = (int(fir_pt_x), int(fir_pt_y))
            end_point = (int(sec_pt_x), int(sec_pt_y))
            bgr_color = hex_to_bgr(line_color)
            cv2.line(img, start_point, end_point, bgr_color, 5)
    return img

In [10]:
def get_color(name):
    if name == 'cow' or name == 'animal':
        color = '#FF9300'
    elif name == 'head' or name == "head left" or name == "head right":
        color = '#4394F9'
    elif name == 'tag':
        color = '#00FFFF'
    elif name == 'knee':
        color = '#FFFB00'
    elif name == 'hoof':
        color = '#00F900'
    elif name == 'tail':
        color = '#FF40FF'
    elif name == 'side left' or name == 'side right':
        color = '#FF2600'
    elif name == 'udder':
        color = '#9437FF'
    elif name == 'teat':
        color = '#FF2F92'
    else:
        color = '#000000'
    return color

def get_bbox_type(label):
    bbox_type = 'head'
    if label == 'Spine' or label == 'Leg_front' or label == 'Leg_back':
        bbox_type = 'cow'
    elif label == 'Tail':
        bbox_type = 'tail'
    return bbox_type

def get_opacity(name):
    if name == 'cow' or name == 'animal':
        opacity = 0.3
    elif name == 'tag':
        opacity = 0.3
    elif name == 'head' or name == "head Left" or name == "head Right":
        opacity = 0.45
    elif name == 'knee':
        opacity = 0.3
    elif name == 'hoof':
        opacity = 0.35
    elif name == 'tail':
        opacity = 0.3
    elif name == 'side left' or name == 'side right':
        opacity = 0.3
    elif name == 'udder':
        opacity = 0.35
    elif name == 'teat':
        opacity = 0.3
    else:
        opacity = 0.0

    return opacity

def hex_to_bgr(color_str):
    # convert color
    hex_color = color_str.lstrip("#")
    # convert outline color
    # bgr: 4, 2, 0 or rgb: 0, 2, 4
    return tuple(int(hex_color[i:i+2], 16) for i in (4, 2, 0))

In [11]:
def draw_bbox(img, bbox, bbox_config, draw_boundary=True, draw_fill=True, draw_btn=True):
    left, top, width, height = bbox['top_left_x'], bbox['top_left_y'], bbox['width'], bbox['height']
    label = get_bbox_type(bbox['label'])
    #draw bbox
#     color = get_color(label)
    color = bbox_config[label]['filled_color']
#     opacity = get_opacity(label)
    opacity = bbox_config[label]['opacity']
    border_width = bbox_config[label]['border_width']
    start_point = (int(left), int(top))
    end_point = (int(left+width), int(top+height))
    bgr_color =  hex_to_bgr(color)
    font = bbox_config[label]['font']
    font_size = bbox_config[label]['font_size']
    if draw_fill:
        # Draw filled bbox
        # Initialize blank mask image of same dimensions for drawing the shapes
        shapes = np.zeros_like(img, np.uint8)
        cv2.rectangle(shapes, start_point, end_point, bgr_color, cv2.FILLED)
        mask = shapes.astype(bool)
        img[mask] = cv2.addWeighted(img, 1-opacity+0.15, shapes, opacity, 0)[mask]
    if draw_boundary:
        img = cv2.rectangle(img, start_point, end_point, bgr_color, border_width)
    if draw_btn:
        end_point = (int(left)+8+len(label)*10, int(top)+20)
        img = cv2.rectangle(img, start_point, end_point, bgr_color, cv2.FILLED)
        img = cv2.putText(img, label, (int(left)+5, int(top)+15), font, font_size, (0,0,0), 1, cv2.LINE_AA)
    if bbox_config[label]['show_conf']:
        bbox_conf = bbox['conf']
        conf_str = "conf: {:.1f}%".format(bbox_conf)
        img = cv2.putText(img, conf_str, (int(left), int(top)-20), font, font_size, (0,0,0), 1, cv2.LINE_AA)

In [12]:
def get_overlap_area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

In [13]:
def draw_logo(img, logo_config):
    #LogoV2_Black.png LogoV2-White_ClearBknd.png
    logo_img = cv2.imread('LogoV2-White_ClearBknd.png', -1)
    x_offset = logo_config['x']
    y_offset = logo_config['y']
    scale_rate =  0.12 # percent of original size
    width = int(logo_img.shape[1] * scale_rate)
    height = int(logo_img.shape[0] * scale_rate)
    dim = (width, height)
    # resize logo image
    logo_img = cv2.resize(logo_img, dim, interpolation = cv2.INTER_AREA)
    # draw transparent background
    start_point = (0, 0)
    end_point = (int(width+x_offset), int(height+y_offset-5))
    bgr_color =  hex_to_bgr(logo_config['background_color'])
    opacity = logo_config['background_opacity']
    shapes = np.zeros_like(img, np.uint8)
    cv2.rectangle(shapes, start_point, end_point, bgr_color, cv2.FILLED)
    cv2.rectangle(shapes, start_point, end_point, bgr_color, 4)
    mask = shapes.astype(bool)
    img[mask] = cv2.addWeighted(img, 1-opacity, shapes, opacity, 0)[mask]

    # draw logo
    y1, y2 = y_offset, y_offset + logo_img.shape[0]
    x1, x2 = x_offset, x_offset + logo_img.shape[1]

    alpha_s = logo_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    for c in range(0, 3):
        img[y1:y2, x1:x2, c] = (alpha_s * logo_img[:, :, c] +
                                  alpha_l * img[y1:y2, x1:x2, c])
    return img

In [14]:
def draw_count(count_cow_bbox, img, count_config):
    x_offset = count_config['x']
    y_offset = count_config['y']
#     width = img.shape[1]
#     height = img.shape[0]
    # draw transparent background
    start_point = (x_offset, 0)
    end_point = (int(x_offset+180), int(y_offset+150))
    bgr_color =  hex_to_bgr(count_config['background_color'])
    opacity = count_config['background_opacity']
    shapes = np.zeros_like(img, np.uint8)
    cv2.rectangle(shapes, start_point, end_point, bgr_color, -1)
    cv2.rectangle(shapes, start_point, end_point, bgr_color, 3)
    mask = shapes.astype(bool)
    img[mask] = cv2.addWeighted(img, 1-opacity, shapes, opacity, 0)[mask]
    cv2.putText(img, "{:02d}".format(count_cow_bbox), (x_offset, 120), 
            count_config['font'], count_config['font_size'], (255, 255, 255), 5, cv2.LINE_AA)
    return img

In [15]:
def cal_cowcard_size(head_bbox, spine_bbox):
    height = spine_bbox['height']*0.33
    width = height*2
    return width * height

In [16]:
def draw_cowcard(head_bbox, spine_bbox, img, cowcard_config):
    height = spine_bbox['height']*0.33
    width = height*2
    tl_x = spine_bbox['top_left_x']
    tl_y = spine_bbox['top_left_y']
    card_start_x = int(tl_x + (spine_bbox['width'] - width) / 2)
    card_start_y = int(tl_y-height-20)
    # card left out of the left bound
    if card_start_x < 0:
#         print("no room for card x")
        card_start_x = 0
    end_x = int(card_start_x+width)
    end_y = int(tl_y-20)
    # card top out of the top bound
    if card_start_y < 0:
#         print("no room for card y")
        card_start_y = int(tl_y + (spine_bbox['height'] - height) / 4)
        end_y = int(card_start_y+height)
    start_point = (card_start_x, card_start_y)
    end_point = (end_x, end_y)
    bgr_color = hex_to_bgr(cowcard_config['background_color'])
    opacity = cowcard_config['background_opacity']
    shapes = np.zeros_like(img, np.uint8)
    # draw background
    cv2.rectangle(shapes, start_point, end_point, bgr_color, cv2.FILLED)
    mask = shapes.astype(bool)
    img[mask] = cv2.addWeighted(img, 1-opacity, shapes, opacity, 0)[mask]
    # draw border
    bgr_color = hex_to_bgr(cowcard_config['border_color'])
    border_width = cowcard_config['border_width']
    cv2.rectangle(img, start_point, end_point, bgr_color, border_width)
    # draw cow img
    margin = 10
    cow_img_w = int((width - margin) * 0.4)
    cow_img_h = int((end_y - card_start_y - margin) * 1)
    cow_img_start = (int(card_start_x+margin), int(card_start_y+margin))
    cow_img_end = (int(card_start_x+cow_img_w), int(card_start_y+cow_img_h))
    cv2.rectangle(img, cow_img_start, cow_img_end, (0, 0, 0), 2)
    # draw cow info
    row_num = 4
    font_size = cow_img_h / 220
    text_h = font_size * 25
    text_margin = (height - (text_h * row_num)) / 6
    cv2.putText(img, "ID: xxxxxxx", (int(card_start_x+cow_img_w)+10, int(card_start_y+text_margin+text_h)), 
                0, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(img, "Tag: xxxxxxx", (int(card_start_x+cow_img_w)+10, int(card_start_y+text_margin*2+text_h*2)), 
                0, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(img, "FaceId: xxxxxxx", (int(card_start_x+cow_img_w)+10, int(card_start_y+text_margin*3+text_h*3)), 
                0, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(img, "Activity: xxxxxxx", (int(card_start_x+cow_img_w)+10,  int(card_start_y+text_margin*4+text_h*4)), 
                0, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    return img

In [17]:
# take second element for sort
def takeSize(elem):
    return elem[2]

def draw_data(img, kp_data, bbox_data, frameId, angle_list, config):
    count_cow_bbox = 0
    # draw bounding box
    for idx, row in bbox_data.iterrows():
        draw_bbox(img, row, config['bbox'])
    spine_pred_df = kp_data[kp_data['label'] == 'Spine'].groupby('bbox_idx')
    tail_pred_df = kp_data[kp_data['label'] == 'Tail'].groupby('bbox_idx')
    legf_pred_df = kp_data[kp_data['label'] == 'Leg_front'].groupby('bbox_idx')
    legb_pred_df = kp_data[kp_data['label'] == 'Leg_back'].groupby('bbox_idx')
    head_pred_df = kp_data[kp_data['label'] == 'Head'].groupby('bbox_idx')
    face_right_list = []
    # draw kp and lines in bbox
    for i, spine_pred in spine_pred_df:
        # draw connection line with diff part
        img = draw_line_connect(spine_pred, legf_pred_df.get_group(i), 'Spine', 'Leg_front', img , config['keypoint'])
        img = draw_line_connect(spine_pred, legb_pred_df.get_group(i), 'Spine', 'Leg_back', img , config['keypoint'])
        points = [i[-3:] for i in spine_pred.values.tolist()]
        face_right = False
        if points[1][0] > points[2][0]:
            face_right = True
        face_right_list.append(face_right)
        # draw kps and angle
        img = vis_pose(spine_pred, False, img, 'Spine', frameId, angle_list, config['keypoint'], config['angle'])
    for i, tail_pred in tail_pred_df:
        img = vis_pose(tail_pred, False, img, 'Tail', frameId, angle_list, config['keypoint'], config['angle'])
    for i, legf_pred in legf_pred_df:
        img = vis_pose(legf_pred, face_right_list[i], img, 'Leg_front', frameId, angle_list, config['keypoint'], config['angle'])
    for i, legb_pred in legb_pred_df:
        img = vis_pose(legb_pred, face_right_list[i], img, 'Leg_back', frameId, angle_list, config['keypoint'], config['angle'])
        count_cow_bbox+=1
    for i, head_pred in head_pred_df:
        img = vis_pose(head_pred, False, img, 'Head', frameId, angle_list, config['keypoint'], config['angle'])
    # spilt to bbox
    spine_bbox_df = bbox_data[bbox_data['label'] == 'Spine'].groupby('bbox_idx')
    head_bbox_df = bbox_data[bbox_data['label'] == 'Head'].groupby('bbox_idx')
    tail_bbox_df = bbox_data[bbox_data['label'] == 'Tail'].groupby('bbox_idx')
    match_list = []
    # search cooresponding bbox for drawing on mutual bbox
    for i, spine_bbox in spine_bbox_df:
        spine_bbox = spine_bbox.iloc[0]
        # find corresponding head bbox and get mutual angle
        for j, head_bbox in head_bbox_df:
            head_bbox = head_bbox.iloc[0]
            ra = Rectangle(spine_bbox['top_left_x'], spine_bbox['top_left_y'], spine_bbox['top_left_x']+spine_bbox['width'], spine_bbox['top_left_y']+spine_bbox['height'])
            rb = Rectangle(head_bbox['top_left_x'], head_bbox['top_left_y'], head_bbox['top_left_x']+head_bbox['width'], head_bbox['top_left_y']+head_bbox['height'])
            overlap_area = get_overlap_area(ra, rb)
            head_area = head_bbox['width'] * head_bbox['height']
            if overlap_area and head_area * 0.2 < overlap_area:
                img = vis_mutual_angle(head_pred_df.get_group(j), spine_pred_df.get_group(i), 'Head', 'Spine', img, frameId, angle_list, config['angle'])
                card_size = cal_cowcard_size(head_bbox, spine_bbox)
                match_list.append([head_bbox, spine_bbox, card_size])
                break
        # find corresponding tail bbox and get mutual angle
        for j, tail_bbox in tail_bbox_df:
            tail_bbox = tail_bbox.iloc[0]
            ra = Rectangle(spine_bbox['top_left_x'], spine_bbox['top_left_y'], spine_bbox['top_left_x']+spine_bbox['width'], spine_bbox['top_left_y']+spine_bbox['height'])
            rb = Rectangle(tail_bbox['top_left_x'], tail_bbox['top_left_y'], tail_bbox['top_left_x']+tail_bbox['width'], tail_bbox['top_left_y']+tail_bbox['height'])
            overlap_area = get_overlap_area(ra, rb)
            tail_area = tail_bbox['width'] * tail_bbox['height']
            if overlap_area and tail_area * 0.2 < overlap_area:
                img = vis_mutual_angle(tail_pred_df.get_group(j), spine_pred_df.get_group(i), 'Tail', 'Spine', img, frameId, angle_list, config['angle'])
                break
    match_list.sort(key=takeSize)
    for bbox_pair in match_list:
        # draw cow box
        img = draw_cowcard(bbox_pair[0], bbox_pair[1], 
                           img, config['bbox']['cow']['cowcard'])
    img = draw_logo(img, config['logo'])
    img = draw_count(count_cow_bbox, img, config['count'])
    return img

In [18]:
def analyzeVideo(src_video, src_data_dir, src_img_dir, output_file, video, fps=5):
    start = time.time()
    srcBGR = cv2.imread(src_img_dir+str(0)+'.jpg')
    height, width, channels = srcBGR.shape
    imgSize = (width, height)
    cap = cv2.VideoCapture(src_video)
    frameRate = cap.get(fps) #frame rate
    print('FrameRate:', frameRate)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(output_file, fourcc, frameRate, imgSize) 
    
    with open('json_data_v3/config.json') as config_json:
        config = json.load(config_json)
        #load filtered rescaled bbox and kps
        kp_df = pd.read_csv(src_data_dir+'kp.csv')
        bbox_df = pd.read_csv(src_data_dir+'bbox.csv')
        print(kp_df.info(verbose=True))
        print(bbox_df.info(verbose=True))
        grouped_bbox_df = bbox_df.groupby('frameId')
        grouped_kp_df = kp_df.groupby('frameId')
        angle_list = []
        for frameId, bbox_data in grouped_bbox_df:
            # skip frame 350, 450, 550, 650, 750, 850, 950
            #550, 850 opposite
            frame_list = [1300, 1350, 1400]
            checking = False
            if checking and frameId not in frame_list:
                continue
            kp_data = grouped_kp_df.get_group(frameId)
            # get each image frame
            srcBGR = cv2.imread(src_img_dir+str(frameId)+'.jpg')
            img = cv2.cvtColor(srcBGR, cv2.COLOR_BGR2RGB)
            anno_img = draw_data(img, kp_data, bbox_data, frameId, angle_list, config)
            # uncommet this part for testing
            if checking and (frameId in frame_list):
                temp_frame = cv2.cvtColor(anno_img, cv2.COLOR_BGR2RGB)
                temp_image = Image.fromarray(temp_frame)
                temp_image.save('./'+'IMG_4195_'+str(frameId)+'_image.jpg')
            # check each 50 frame
            if frameId % 50 == 0:
                print("Finish Processing {} frame".format(frameId))
                lap = time.time()
                print('lap time: ', lap - start)
            videoWriter.write(anno_img)
        angle_df = pd.DataFrame.from_records(angle_list, columns=['frameId', 'keypoint_type', 'kp_x', 'kp_y', 'angle'])
        print(angle_df.info(verbose=True))
        angle_df.to_csv('csv_data/'+video+'_angle.csv',index=False)
    videoWriter.release()
    cv2.destroyAllWindows()
    config_json.close()
    
    #end time
    end = time.time()
    print('total time lapse', end - start)

In [19]:
# main function
video_name_list = ['cattle_multi_1']
video_format = ['.mov']
for v_idx, video in enumerate(video_name_list):
    src_video = 'video_data/input_video/'+video+video_format[v_idx]
    src_data_dir = 'csv_data/'+video+'_'
    src_img_dir = 'frame_img/'+video+'/'
    output_video = 'video_data/inferred_video/inferred_v3_'+video+'.mp4'
    print('output_video:', output_video)
    analyzeVideo(src_video, src_data_dir, src_img_dir, output_video, video)
    print('finished analyzing the video '+video)
    print()

output_video: video_data/inferred_video/inferred_v3_cattle_multi_1.mp4
FrameRate: 30.006466910972193
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378207 entries, 0 to 378206
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   frameId   378207 non-null  int64  
 1   label     378207 non-null  object 
 2   bbox_idx  378207 non-null  int64  
 3   kp_idx    378207 non-null  int64  
 4   x         378207 non-null  float64
 5   y         378207 non-null  float64
 6   conf      378207 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 20.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19963 entries, 0 to 19962
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   frameId     19963 non-null  int64  
 1   label       19963 non-null  object 
 2   bbox_idx    19963 non-null  int64  
 3   top_left_x  19963 non-null  float64
 4   to